# P01 - 08: Introducing `scipy`

**April 23, 2020**  
*Ingo Scholtes*  

This notebook introduces basic statistical concepts in the package `scipy`, which we will use for statistical data analysis and hypothesis in the following practice lectures.

## Sparse matrix algebra

Naturally extending the introduction of vector and matrix operations in `numpy` in the last unit, we first highlight `scipy`'s support for efficient operations on sparse matrices and vector, i.e. large matrices where most entries are actually zeros. In data science, we often deal with such matrices, be it in the context of network analysis and graph mining (where each non-existing link leads to a zero entry in the adjacency matrix) or when we we deal with matrices capturing similarity scores (where zero entries indicate objects that are either not similar or for which the similarty is not known).

Clearly, it is not efficient to perform mathematical operations on matrices where the majority of entries is zero anyway. The module `scipy.sparse` thus offers classes to represent such matrices. The class `csr_matrix` (CSR = Compressed Sparse Row) for instance can be initialised from a list of `python` lists or, conveniently, from a `numpy` array. As we see below, different from a `numpy` array, the class `csr_matrix` only stores the non-zero entries of the matrix:

In [1]:
import numpy as np
import scipy.sparse

matrix = np.array([[1., 0., 1.], [0., 1., 0.], [0., 0., 1.]])
print(matrix)

sparse_matrix = scipy.sparse.csr_matrix(matrix)
sparse_matrix
print(sparse_matrix)

[[1. 0. 1.]
 [0. 1. 0.]
 [0. 0. 1.]]
  (0, 0)	1.0
  (0, 2)	1.0
  (1, 1)	1.0
  (2, 2)	1.0


If we wish to recover a representation with zeros, we can call the `to_dense` method on an instance of `csr_matrix`. This function returns an instance of the class `numpy.matrix`:

In [2]:
print(sparse_matrix.todense())
print(type(sparse_matrix.todense()))

[[1. 0. 1.]
 [0. 1. 0.]
 [0. 0. 1.]]
<class 'numpy.matrix'>


We can directly perform vector and matrix operations on sparse matrices, i.e. we can do the following:

In [3]:
v = scipy.sparse.csr_matrix([2,3,1])
w = v.dot(sparse_matrix)
print(w.todense())

[[2. 3. 3.]]


To compute eigenvalues and eigenvectors, we can use the `eigs` function:

In [4]:
import scipy.sparse.linalg

w, v = scipy.sparse.linalg.eigs(sparse_matrix, k=1)
print(w)
print(v)

[1.+0.j]
[[ 2.65526779e-04+0.j]
 [-9.99999965e-01+0.j]
 [-1.77839385e-17+0.j]]


In [5]:
print(sparse_matrix.dot(v[:,0]))
print(w[0]* v[:,0])

[ 2.65526779e-04+0.j -9.99999965e-01+0.j -1.77839385e-17+0.j]
[ 2.65526779e-04+0.j -9.99999965e-01+0.j -1.77839385e-17+0.j]


## Statistical computing with `scipy.stats`

In our course, we will extensively use `scipy`'s implementation of statistical functions, random variables, and probability distributions to generate random multi-variate data, perform linear regression, or test hypotheses. These functions are implemented in the statistics module `scipy.stats`, which contains an [extensive list of classes](https://docs.scipy.org/doc/scipy/reference/stats.html) that implements random variables distributed according various well-known probability distributions. We can, for instance, create a continuous random variable distributed according to a standard normal distribution (with mean 0 and standard deviation 1) as follows:





In [6]:
import scipy.stats
norm = scipy.stats.norm()

Depending on the domain of the distribution, `scipy` returns an instance of class [`rv_continuous`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.rv_continuous.html#scipy.stats.rv_continuous) or [`rv_discrete`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.rv_discrete.html#scipy.stats.rv_discrete). Both classes provide functions that -- among other things -- allow us to (i) generate random realisations of the random variable, (ii) calculate moments of the distribution (i.e. mean, variance, skewness, etc.), and (iii) get the underlying probability distribution or cumulative distribution function.

To generate a `numpy` array of 100 random realisations of the normally distributed random variable, we can do the following:

In [7]:
x = scipy.stats.norm.rvs(size=100)
print(x)

[ 0.13534168 -1.00952803  0.77448814  0.6710512  -0.89283774 -0.314427
 -0.77645475  0.645485    0.24913613  0.61844554 -0.00395505 -0.53668015
  0.50233061 -0.67485157  2.89800895 -1.60677911  1.08209847  0.30794145
 -2.19027266 -1.71620504  0.03569584 -1.55183238  0.01985153  0.05459907
  0.89135492  0.56636003  0.55741042 -0.62882403  1.15309218  1.16990735
 -0.74348215  0.3186753   1.14738583 -0.37393937  1.602499   -1.4242566
  0.1130705   1.15333922 -0.30012787 -0.68281587  0.83081104  0.61176285
 -0.72713452  0.53478174 -0.38956851 -1.53914415  1.36492859 -1.33182518
 -1.20548287 -1.53856484  0.81915352 -0.00798128 -0.982206    0.76314186
  1.21280394 -0.79784743 -1.27806637  1.76702113  0.80135559  0.53328068
  0.16919552 -0.21082889  0.22855603 -1.04151423 -1.36382629  1.25784819
  0.23470321 -0.30216807 -0.66064698  0.05528471  1.09671789 -0.28980808
 -0.57543303  0.06346526 -0.20519158  0.66509537 -0.17655788 -1.54037718
 -0.34961678  0.87125037 -0.27259372 -1.53199103  1.35

The above call generates 100 realisations of a normally distributed random variable with mean $\mu=0$ and standard deviation $\sigma^2=1$. A simple way to create realisations drawn from a normal distribution with arbitrary $\mu$ and $\sigma$ is to perform an element-wise addition and multiplication of the resulting `numpy` array. To obtain realisations from a normal distribution with mean $\mu=5$ and standard deviation $\sigma=2$, we can thus write:

In [8]:
x = 5 + 2 * scipy.stats.norm.rvs(size=5000)
print('Sample mean = {0}'.format(np.mean(x)))
print('Sample variance = {0}'.format(np.std(x)))

Sample mean = 4.9802897815288265
Sample variance = 2.0218355010460876


Let us try this with a discrete random variable, that is distributed according to the Poisson distibution. We can directly pass the single parameter $\mu$ of the distribution to the `rvs` function.

In [9]:
x = scipy.stats.poisson.rvs(mu=5, size=100)
print(x)

[6 6 7 6 8 3 5 3 7 2 6 2 3 3 6 5 1 5 5 6 9 6 3 5 4 6 4 7 6 4 3 4 4 4 5 8 5
 5 6 6 3 4 4 5 7 7 2 8 4 5 4 6 5 3 8 6 7 8 8 3 5 3 6 4 4 5 4 8 4 4 3 5 1 6
 4 6 5 5 5 6 5 4 3 2 2 8 5 7 5 9 4 6 5 5 8 2 5 8 5 4]


The following generates 100 realisations of a random variable distributed according to a Binomial distribution with parameters $n$ and $k$. For $n=1$ we obtain a sequence of Bernoulli trials that can be 0 or 1.

In [10]:
x = scipy.stats.binom.rvs(p=0.5, n=1, size=100)
print(x)

[1 0 0 1 1 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 1 0 1 1 1 1 1 0 1 1 1 0 1 0 1 0
 1 0 1 0 0 0 0 1 0 1 0 1 1 1 1 1 0 1 1 1 1 1 1 0 0 1 0 1 0 0 0 1 1 0 0 0 1
 0 0 1 1 1 0 1 1 0 1 1 0 0 0 1 0 1 0 1 1 0 1 1 1 0 1]


For continuous random variables, we can obtain the value of the probability distribution or the cumulative distribution function at any point $x$ as follows:

In [11]:
x = scipy.stats.norm.pdf(x=0.1)
print(x)

x = scipy.stats.norm.cdf(x=0.1)
print(x)

0.3969525474770118
0.539827837277029


For a discrete random variable, we can directly calculate the probability of any specific value based on the probability mass function:

In [12]:
x = scipy.stats.binom.pmf(p=0.5, n=10, k=9)
print(x)

0.00976562500000001
